# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

In [1]:
# Haciendo la funcion:
def mi_funcion_chula(game_df,game_column):# (nhl_df,"NHL")
        
    global cities
    
    # Seleccionando el año a analizar:
    game_df = game_df[game_df["year"] == 2018]
    
    # Haciendo limpieza y manejo de datos del dataframe a analizar:
    game_df = game_df[game_df.iloc[:,0]!=game_df.iloc[:,1]]
    game_df["team"] = game_df["team"].replace({"\s\(\d{1,2}\)":""}, regex=True) # Elimina espacio y paréntesis    
    game_df["team"] = game_df["team"].replace({"\+|\*":""}, regex=True) # Elimina signo + o *
    #game_df["team"] = game_df["team"].replace({"[^\w ]":""}, regex=True) # Es más general que la anterior
    game_df[["W","L"]] = game_df[["W","L"]].apply(pd.to_numeric)
    game_df["win/loss"] = game_df["W"]/(game_df["W"]+game_df["L"])
    
    # Haciendo limpieza y manejo de datos del dataframe "Cities"
    cities = cities.replace({"\[[\w ]*\]":""}, regex=True)
    cities["Population (2016 est.)[8]"] = cities["Population (2016 est.)[8]"].apply(pd.to_numeric)
    
    # Agregando las columnas necesarias para la correlación al dataframe a analizar:
    metropolitan_area = []
    population = []
    for team in game_df["team"]:
        for teams in cities[game_column]:
            if team.split(" ")[-1] in teams:
                row = cities[cities[game_column] == teams]
                metropolitan_area.append(cities["Metropolitan area"].iloc[row.index].iloc[0])
                population.append(cities["Population (2016 est.)[8]"].iloc[row.index].iloc[0])
    game_df["Metropolitan area"] = metropolitan_area
    game_df["Population (2016 est.)[8]"] = population
    
    #Hacemos un merge y lo ajustamos al orden del df "cities" con solo las columnas que nos interesan:
    the_df = (pd.DataFrame.merge(cities, right=game_df, how="left", on="Metropolitan area")
                 .dropna()[["Metropolitan area","Population (2016 est.)[8]_x","win/loss"]])
    
    # El DataFrame resultante tiene 2 columnas para ser utilizadas en la correlación.
    result = the_df.groupby("Metropolitan area", sort=False).agg({"Population (2016 est.)[8]_x": np.mean, "win/loss": np.mean})
    
    return result

# La funcion debe retornar el df con las nuevas columnas

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("utf-8''nhl.csv")
cities=pd.read_html("utf-8''wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    df = mi_funcion_chula(nhl_df,"NHL")
    #raise NotImplementedError()
    
    population_by_region = df["Population (2016 est.)[8]_x"] # enter metropolitan area population from cities
    win_loss_by_region = df["win/loss"] # enter win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

In [3]:
nhl_correlation()

(0.012486162921209912, 0.9497182859911781)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("utf-8''nba.csv")
cities=pd.read_html("utf-8''wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    df = mi_funcion_chula(nba_df,"NBA")
    #raise NotImplementedError()
    
    population_by_region = df["Population (2016 est.)[8]_x"] # pass in metropolitan area population from cities
    win_loss_by_region = df["win/loss"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [5]:
nba_correlation()

(-0.17657160252844617, 0.36874741604463)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("utf-8''mlb.csv")
mlb_df.iloc[0,0] = 'Red_Sox'
mlb_df.iloc[8,0] = 'White_Sox'
cities=pd.read_html("utf-8''wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.iloc[3,3] = "CubsWhite_Sox"
cities.iloc[7,3] = "Red_Sox"
def mlb_correlation(): 
    df = mi_funcion_chula(mlb_df,"MLB")
    #raise NotImplementedError()
    
    population_by_region = df["Population (2016 est.)[8]_x"] # pass in metropolitan area population from cities
    win_loss_by_region = df["win/loss"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    return stats.pearsonr(population_by_region, win_loss_by_region)

In [7]:
mlb_correlation()

(0.1502769830266931, 0.46370703378875605)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("utf-8''nfl.csv")
cities=pd.read_html("utf-8''wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    df = mi_funcion_chula(nfl_df,"NFL")
    #raise NotImplementedError()
    
    population_by_region = df["Population (2016 est.)[8]_x"] # pass in metropolitan area population from cities
    win_loss_by_region = df["win/loss"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [9]:
nfl_correlation()

(0.004922112149349386, 0.979783345836372)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [16]:
# no la tuve bien :(

In [10]:
# Haciendo la funcion:
def mi_funcion_chula_mod(game_df,game_column):# (nhl_df,"NHL")
        
    global cities
    
    # Seleccionando el año a analizar:
    game_df = game_df[game_df["year"] == 2018]
    
    # Haciendo limpieza y manejo de datos del dataframe a analizar:
    game_df = game_df[game_df.iloc[:,0]!=game_df.iloc[:,1]]
    game_df["team"] = game_df["team"].replace({"\s\(\d{1,2}\)":""}, regex=True) # Elimina espacio y paréntesis    
    game_df["team"] = game_df["team"].replace({"\+|\*":""}, regex=True) # Elimina signo + o *
    #game_df["team"] = game_df["team"].replace({"[^\w ]":""}, regex=True) # Es más general que la anterior
    game_df[["W","L"]] = game_df[["W","L"]].apply(pd.to_numeric)
    game_df["win/loss"] = game_df["W"]/(game_df["W"]+game_df["L"])
    
    # Haciendo limpieza y manejo de datos del dataframe "Cities"
    cities = cities.replace({"\[[\w ]*\]":""}, regex=True)
    cities["Population (2016 est.)[8]"] = cities["Population (2016 est.)[8]"].apply(pd.to_numeric)
    
    # Agregando las columnas necesarias para la correlación al dataframe a analizar:
    metropolitan_area = []
    population = []
    for team in game_df["team"]:
        for teams in cities[game_column]:
            if team.split(" ")[-1] in teams:
                row = cities[cities[game_column] == teams]
                metropolitan_area.append(cities["Metropolitan area"].iloc[row.index].iloc[0])
                population.append(cities["Population (2016 est.)[8]"].iloc[row.index].iloc[0])
    game_df["Metropolitan area"] = metropolitan_area
    game_df["Population (2016 est.)[8]"] = population
    
    #Hacemos un merge y lo ajustamos al orden del df "cities" con solo las columnas que nos interesan:
    the_df = (pd.DataFrame.merge(cities, right=game_df, how="left", on="Metropolitan area")
                 .dropna()[["Metropolitan area",game_column,"win/loss"]])
    
    # El DataFrame resultante tiene 2 columnas para ser utilizadas en la correlación.
    #result = the_df.groupby("Metropolitan area", sort=False).agg({"Population (2016 est.)[8]_x": np.mean, "win/loss": np.mean})
    
    return the_df

# La funcion debe retornar el df con las nuevas columnas

In [11]:
mi_funcion_chula_mod(nhl_df,"NHL")

,Metropolitan area,NHL,win/loss
0,New York City,RangersIslandersDevils,0.602740
1,New York City,RangersIslandersDevils,0.486111
2,New York City,RangersIslandersDevils,0.465753
3,Los Angeles,KingsDucks,0.637681
4,Los Angeles,KingsDucks,0.608108
5,San Francisco Bay Area,Sharks,0.625000
6,Chicago,Blackhawks,0.458333
7,Dallas–Fort Worth,Stars,0.567568
8,"Washington, D.C.",Capitals,0.653333
9,Philadelphia,Flyers,0.617647


In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("utf-8''mlb.csv")
mlb_df.iloc[0,0] = 'Red_Sox'
mlb_df.iloc[8,0] = 'White_Sox'
nhl_df=pd.read_csv("utf-8''nhl.csv")
nba_df=pd.read_csv("utf-8''nba.csv")
nfl_df=pd.read_csv("utf-8''nfl.csv")
cities=pd.read_html("utf-8''wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.iloc[3,3] = "CubsWhite_Sox"
cities.iloc[7,3] = "Red_Sox"

def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [15]:
sports_team_performance()

AssertionError: The NBA-NHL p-value should be around 0.02